In [ ]:
import paho.mqtt.client as mqtt

# --- การตั้งค่าตามรูปภาพ ---
BROKER_HOST = "100.85.77.73"
BROKER_PORT = 1883  # พอร์ตมาตรฐานของ mqtt / tcp
USERNAME = "yessuskhonpui"
PASSWORD = "246810"  # !! ต้องใส่รหัสผ่านจริงของคุณที่นี่ !!
KEEPALIVE = 10
SUB_TOPIC = "to"  # Topic ที่จะ Subscribe (จากช่อง Will - Topic)

# !! ข้อสำคัญ: Client ID ต้องไม่ซ้ำกัน !!
# MQTT Broker จะตัดการเชื่อมต่อ Client ที่มี ID ซ้ำกัน
# ผมจึงตั้งชื่อใหม่ไม่ให้ซ้ำกับใน MQTTBox
CLIENT_ID_SUB = "python-subscriber-01" 

# --- Callback Functions ---

def on_connect(client, userdata, flags, rc, properties=None):
    """
    ถูกเรียกเมื่อเชื่อมต่อกับ Broker สำเร็จ
    """
    if rc == 0:
        print(f"เชื่อมต่อ Broker {BROKER_HOST} สำเร็จ!")
        # เมื่อเชื่อมต่อสำเร็จ ให้ Subscribe ไปยัง Topic ที่กำหนด
        client.subscribe(SUB_TOPIC)
        print(f"กำลังดักฟังข้อความบน Topic: '{SUB_TOPIC}'")
    else:
        print(f"เชื่อมต่อไม่สำเร็จ, รหัสข้อผิดพลาด: {rc}")

def on_message(client, userdata, msg):
    """
    ถูกเรียกทุกครั้งที่ได้รับข้อความ
    """
    print(f"ได้รับข้อความ: '{msg.payload.decode()}' บน Topic: '{msg.topic}'")

# --- Main Script ---

# 1. สร้าง Client instance
#    (ใช้ Callback API v2 และกำหนด Client ID ที่ไม่ซ้ำ)
client = mqtt.Client(client_id=CLIENT_ID_SUB, 
                     callback_api_version=mqtt.CallbackAPIVersion.VERSION2)

# 2. ตั้งค่า Username และ Password
client.username_pw_set(USERNAME, PASSWORD)

# 3. ตั้งค่า Last Will (ตามในรูป)
#    Topic: "to", QoS: 0, Retain: No, Payload: (ว่างเปล่า)
client.will_set(SUB_TOPIC, payload=None, qos=0, retain=False)

# 4. ผูกฟังก์ชัน Callback
client.on_connect = on_connect
client.on_message = on_message

# 5. เชื่อมต่อ
try:
    print(f"กำลังเชื่อมต่อไปยัง {BROKER_HOST}...")
    client.connect(BROKER_HOST, BROKER_PORT, keepalive=KEEPALIVE)
except Exception as e:
    print(f"ไม่สามารถเชื่อมต่อได้: {e}")
    exit() # ออกจากโปรแกรมถ้าเชื่อมต่อไม่ได้

# 6. เริ่ม network loop (แบบ blocking)
#    สคริปต์จะทำงานค้างไว้ที่บรรทัดนี้เพื่อรอรับข้อความ
print("Subscriber กำลังทำงาน, กด CTRL+C เพื่อออก")
client.loop_forever()

In [1]:
import requests
import os

# --- ตั้งค่า ---
SERVER_IP = "100.85.77.73"
LOCAL_FILE_PATH = "data_to_send.txt"   # <--- ไฟล์บน Jetson ที่คุณต้องการส่ง
REMOTE_FILE_NAME = "jetson_data.txt"   # <--- ชื่อไฟล์ที่จะไปอยู่บน Server
# ---------------

# (ถ้ายังไม่มีไฟล์) สร้างไฟล์ทดสอบขึ้นมาก่อน
if not os.path.exists(LOCAL_FILE_PATH):
    print(f"สร้างไฟล์ทดสอบ '{LOCAL_FILE_PATH}'...")
    with open(LOCAL_FILE_PATH, 'w') as f:
        f.write("Test data from Jetson using Python!")

# URL ปลายทาง
url = f"https://{SERVER_IP}/{REMOTE_FILE_NAME}"

# ปิดการแจ้งเตือนเรื่อง SSL (เพราะเราใช้ Self-signed)
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

try:
    # อ่านไฟล์แบบ binary (rb)
    with open(LOCAL_FILE_PATH, 'rb') as f:
        file_content = f.read()

    # ส่งไฟล์โดยใช้เมธอด PUT
    # verify=False คือสิ่งเดียวกับ -k ใน curl (ยอมรับ Self-signed SSL)
    response = requests.put(url, data=file_content, verify=False)

    # ตรวจสอบว่าสำเร็จหรือไม่
    response.raise_for_status() # ถ้าล้มเหลว (เช่น 404, 500) จะโยน exception

    print(f"สำเร็จ! อัปโหลดไฟล์ไปที่: {url}")
    print(f"Status Code: {response.status_code}") # ควรได้ 201 (Created)

except requests.exceptions.ConnectionError:
    print(f"Error: ไม่สามารถเชื่อมต่อเซิร์ฟเวอร์ที่ {SERVER_IP} ได้")
except requests.exceptions.HTTPError as e:
    print(f"HTTP Error: {e}")
except Exception as e:
    print(f"เกิดข้อผิดพลาด: {e}")

สร้างไฟล์ทดสอบ 'data_to_send.txt'...
สำเร็จ! อัปโหลดไฟล์ไปที่: https://100.85.77.73/jetson_data.txt
Status Code: 201
